# Hyperparameter Tuning
알고리즘을 미세 조정하여 높은 성능의 머신 러닝 모델을 만들고 성능을 평가하는 방법을 기술한다.
- 머신러닝 모델 성능 평가하기
- 머신러닝 알고리즘에서 일반적으로 발생하는 문제 분석하기
- 머신러닝 모델 세부 튜닝하기
- 여러가지 성능 지표를 사용하여 모델의 예측 성능 평가하기

## 파이프라인을 사용한 Workflow
테스트 데이터셋에 있는 별도의 샘플처럼 새로운 데이터의 스케일을 조정하고 압축하기 위해 훈련 데이터셋에서 학습한 파라미터를 재사용해야한다. 이를 위해 sklearn의 Pipeline 클래스를 여기서 기술한다. 이 도구를 사용하여 여러 개의 변환 단계를 포함한 모델을 학습하고 새로운 데이터에 대한 예측을 만들 수 있다.

### 위스콘신 유방암 데이터셋
이 데이터는 악성과 양성인 종양 세포 샘플이 포함되어 있다. 데이터셋의 첫 두열에는 샘플의 고유 ID번호와 진단결과 (M=악성,B=양성)가 들어있습니다. 3번째부터 32번째까지는 이미지에서 추출한 특성이 실수 값으로 들어있습니다.

In [2]:
# pandas를 활용하여 데이터를 로드합니다.
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [5]:
# LabelEncoder를 사용하여 정수로 변환.
from sklearn.preprocessing import LabelEncoder
X = df.loc[:,2:].values
y = df.loc[:,1].values
le = LabelEncoder()
y = le.fit_transform(y)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=1)

### 파이프라인으로 변환기와 추정기 연결

StandardScaler, PCA, LogisticRegression 객체를 하나의 파이프라인으로 구성하는 예제입니다.

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state=1))

pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
print('테스트 정확도: ', pipe_lr.score(X_test,y_test))

테스트 정확도:  0.956140350877193


## K-fold cross-validation
적절한 bias-variance(편향-분산) trade-off를 찾으려면 모델을 주의 깊게 평가해야합니다. 이 절에서는 보편적인 교차검증 기법 **Holdout Cross-Validation**과 **K-fold cross-Validation**을 서술합니다.

### Holdout cross validation
지금까지 해왔던 방식으로 데이터를 학습셋/테스트셋으로 나누어서 학습을 진행하고, parameter Tuning을 하게되면 결국 테스트셋으로 학습을 하는 결과와 비슷한 결과가 나오게 됩니다. 테스트셋으로 모델 선택($\approx$Tuning)이 진행되어서 테스트셋에 과적합 된 모델이 나오게 됩니다.

이러한 문제를 방지하기위해, 데이터를 학습셋/검증셋/테스트셋 으로 나누어 학습과 모델선택, 결과 확인에 사용합니다. 훈련과 모델선택에서 사용된 적 없는 데이터를 사용하여 모델을 평가하기 때문에 일반화 능력을 덜 편향되게 추정할 수 있는 장점이 있습니다.

### K-fold cross-validation
k-겹 교차검증은 중복을 허용하지 않고, 훈련 데이터셋을 k개의 Fold로 랜덤하게 나눕니다. k-1개의 폴드로 모델을 훈련하고 나머지 하나의 폴드로 성능을 평가합니다. 그다음 서로 다른 독립적은 폴드에서 얻은 성능 추정을 기반으로 모델의 평균 성능을 계산합니다. 일반적으로 모델 튜닝에 k-fold 교차검증을 사용합니다. 만족스러운 하이퍼파라미터를 찾은 후에는 전체 훈련 데이터셋(Not folded)을  사용하여 모델을 다시 훈련합니다. k-fold 교차검증 후에 전체 훈련 데이터셋으로 모델을 학습하는 이유는 훈련 샘플이 많을수록 알고리즘이 정확하고 안정적인 모델을 만들기 때문입니다.

일반적으로 10-fold 교차검증이 가장 뛰어난 bias-variance(편향-분산) trade-off를 가진다고 한다.
데이터가 적을 때는 Fold를 늘리고, 데이터가 많다면 5-fold 도 유의미한 성능을 가진다. fold가 줄어들면 학습하고 평가하는 계산 비용을 줄일 수 있습니다.